# Limpieza y Arreglo de Datos 

En este cuaderno se detalla el proceso de limpieza y arreglo del conjuntos de datos: *items_ordered_2years.csv*. El objetivo es normalizar el dataset para que posteriormente se pueda usar en la herramienta sin provocar errores.

## Índice 

1. Importación paquetes
2. Lectura de Datos
3. Ciudades y Códigos Postales
4. Duplicados
5. Fecha y Hora
7. Unificar tablas
8. Precios
9. Conclusiones

## Importación de paquetes

Comenzamos como siempre con la importación de los módulos para la limpieza de datos.

In [110]:
import pandas as pd
import numpy as np

# Semilla para realizar experimentos reproducibles
seed = 126

## Lectura de Datos

Cargamos en memoria el ficheros y comprobamos que se hayan hecho correctamente.

In [111]:
df_items = pd.read_csv('../Data/items_ordered_2years.txt', sep='|', on_bad_lines='skip',parse_dates=['created_at'])
df_items.sample(5,random_state=seed)

,num_order,item_id,created_at,product_id,qty_ordered,base_cost,price,discount_percent,customer_id,city,zipcode
885283,3a9515af945c45aa199a6ffe89a06851,77a7a7c749da68f31204e1bcbebfadbc,2018-12-19 17:33:16,3922,1,6.7000,8.23,5.0,b5aa9f33f1ae3ebd21980a3003ee2a12,BORRIANA,12530
671427,3f816f518b186d384287a55e1e2da020,a581589f2050829d5d16ac38455ee3d2,2018-09-10 16:12:00,13213,1,4.9875,7.00,5.0,668b6e36b0383ec792e3e8a12c56cd5c,ALICANTE,03015
99899,f43bee7f3784e48a2c31e4f83d5e1e70,e72de197dca9bee00fbe702af166fc54,2017-04-25 21:47:15,2731,2,13.6453,18.97,8.0,ac27b77292582bc293a51055bfc994ee,madrid,28040
129128,4dc71dc51db715835e4348f9760c4b07,593b1dc90b7c9da89c1fff2e370dd5b2,2017-06-20 20:26:39,58136,1,7.5900,9.08,10.0,eb8f27d2170d65a37eb72d4b005259ce,cartagena,30300
205362,45381dc59cd2805782eaf54399cf4146,26f5a246442e17a88314457043ca485e,2017-09-04 20:14:42,71146,2,5.0044,7.42,5.0,e4296adfba6141fa05d731b6afd20ead,Villaviciosa de Odon,28670


In [112]:
df_products = pd.read_csv('./clean_products.csv')
df_products.sample(5,random_state=seed)

,product_id,sku,name,marca_value,short_description,analytic_category
24312,84880,WVS128101,victory mass gainer sabor chocolate weider 2 kg,Weider,suplemento alimenticio con 4 fuentes de carboh...,Nutrición
24999,4293,167921,bioderma sebium exfoliating 0 ml gel,Bioderma,"limpiador exfoliante, purificante, alisante y ...",Higiene
18508,86125,187170,masajeador personal inalábrico remote wireless...,Control,control vibrador personal inalámbrico e imperm...,Vida Íntima
20779,90576,ZAM02-15665,aceite de masaje ziaja 500ml,Ziaja,"aceite de masaje anticelulítico, que suaviza l...",Cosmética y Belleza
22829,89514,BHU008-10,esmalte uñas duradero hipoalergénico hypo bell...,Bell,esmalte de uñas hipoalergénico y de larga dura...,Cosmética y Belleza


## Ciudades y Códigos Postales

En este apartado se trataran los campos referentes a la ubicación de los pedidos

In [113]:
# Listamos los campos nulos -> EDA
df_items.isna().sum()

num_order              0
item_id                0
created_at             0
product_id             0
qty_ordered            0
base_cost           2402
price                  0
discount_percent       0
customer_id            0
city                3007
zipcode             3004
dtype: int64

Las columnas de ciudad y código postal tienen sorpendentemente un número muy parecido de registros nulos, por lo que intuimos que de alguna manera están entrelazadas.

In [114]:
# Recuento de ciudades antes de normalizar
print(f"Recuento ciudad únicas: {df_items['city'].nunique()}")

Recuento ciudad únicas: 20993


In [115]:
# Eliminamos registros si ambos son nulos
indexes = df_items[(df_items['city'].isna()) & (df_items['zipcode'].isna())].index
df_items.drop(index=indexes,inplace=True)

In [116]:
df_items.isna().sum()

num_order              0
item_id                0
created_at             0
product_id             0
qty_ordered            0
base_cost           2399
price                  0
discount_percent       0
customer_id            0
city                  97
zipcode               94
dtype: int64

Con todo esto, el número de nulos se ve reducido. Aun así quedan algunos registros nulos, pero no es algo *"preocupante"* ya que con uno de los dos campos es posible recuperar el valor perdido.

In [117]:
# Imputamos zipcode nulos con string vacío
df_items['zipcode'].fillna('',inplace=True)
df_items['city'].fillna('',inplace=True)

In [118]:
print(f"Ciudades con dígitos: {len(df_items[df_items['city'].str.isnumeric()])}")

Ciudades con dígitos: 801


Como ya vimos en el análisis exploratorio, existen ciudad *numéricas* y zipcodes con nombres de ciudades. Tenemos que arreglar este cambio de campos 

In [119]:
# Si ambos campos son numéricos, tan solo nos quedamos con el zipcode y el city lo imputamos con string vacía
indexes = df_items[(df_items['city'].str.isnumeric()) & (df_items['zipcode'].str.isnumeric())].index
df_items.loc[indexes,'city'] = ''

In [120]:
print(f"Ciudades con dígitos: {len(df_items[df_items['city'].str.isnumeric()])}")

Ciudades con dígitos: 356


In [121]:
# Invertimos los valores de los campos en los que se han traspuesto
indexes = df_items[(df_items['city'].str.isnumeric()) & (~df_items['zipcode'].str.isnumeric())].index
df_items.loc[indexes,['city','zipcode']] = df_items.loc[indexes,['zipcode','city']].values

In [122]:
print(f"Ciudades con dígitos: {len(df_items[df_items['city'].str.isnumeric()])}")

Ciudades con dígitos: 0


Otra fuente de problemas es la falta de normalización de los nombres de las ciudades, lo que puede llevar a un mal recuento.

In [123]:
# Ahora la ciudades no tendrán espacios sobrantes y serán escritos en minúsculas
df_items.loc[:, 'city'] = df_items['city'].apply(lambda r: str(r).strip().lower())
df_items[['num_order','city','zipcode']].sample(5,random_state=seed)

,num_order,city,zipcode
691236,9567deb340d90ef2334a8e0eeebb1023,linares,23700
326024,479e26c5bf7c16af391527e4817c560d,carcaixent,46740
412990,fa4759766ef859b1441fc92b27a4efb0,madrid,28008
111622,57d9e0b08fa153794180541d0cf26a98,puerto de mazarron,30860
100077,435e6dfdf710c506307813f27efe7b88,valladolid,47008


In [124]:
# recuento de ciudad tras normalizar
print(f"Recuento ciudad únicas: {df_items['city'].nunique()}")

Recuento ciudad únicas: 11471


Con el fin de mejorar los datos de localización de los pedidos, hemos creado un script con el que extraemos de *Geopy* las coordenadas, la ciudad, la comunidad y el país. Para ello en un primer momento, usamos el *código postal*, aun así esto no es perfecto con tan poca información. Por lo que para reducir los valores nulos que devolvía la API, volvimos a hacer otra tanda de peticiones con una *query* que se basaba en la concatenación del nombre de la ciudad y el código postal. Así reducimos aun más el porcentaje de valores no encontrados, pudiendo *enriquecer* en parte el conjunto de datos y poder crear visualizaciones de mapas.

In [125]:
# Guardamos los zipcodes para utilzar el script de Geopy
zipcodes = df_items['zipcode'].unique().tolist()
df_zipcodes = pd.DataFrame({'zipcode': zipcodes})
df_zipcodes.to_csv('./zipcodes.csv',index=False)

In [126]:
df_geo_1 = pd.read_csv('./city_info.csv', index_col=None)
df_geo_1.isna().sum()

zipcode        1
location     986
coords       986
city        6882
state       2032
country     1084
dtype: int64

De la primera tanda de peticiones, que tardó unas 3 horas en completarse, hay 986 códigos postales de los que no se ha podidos recuperar su ubicación. 

In [127]:
df_geo_2 = pd.read_csv('./query.csv', index_col=None)
df_geo_2.isna().sum()

city           0
zipcode        0
query          0
location     487
coords       487
city_y      1364
state       1688
country      487
dtype: int64

Con la segunda tanda, cambiando la *query*, se pudo disminuir ese número. Procedemos a quedarnos con los valores que nos interesan y a enriquecer nuestro dataset.

In [128]:
df_geo_1.dropna(subset=['location','coords'],inplace=True)
df_geo_2.dropna(subset=['location','coords'],inplace=True)

In [129]:
# Eliminamos filas del primer dataframe que contenga zipcodes del segundo.
# Recordar que la primera tanda engloba también la búsqueda de la segunda tanda
zipcodes = df_geo_2['zipcode'].unique().tolist()
indexes = df_geo_1[df_geo_1['zipcode'].isin(zipcodes)].index 
df_geo_1.drop(index=indexes, inplace=True)

In [130]:
#Quitamos las columnas diferentes (query no está en el otros csv, city_y contiene muchos nulos)
df_geo_2.drop(columns=['query', 'city_y'],inplace=True)

#Concatenamos los dataframes
df_cities = pd.concat([df_geo_1, df_geo_2], axis=0,ignore_index=True)
df_cities.sample(5,random_state=seed)

,zipcode,location,coords,city,state,country
8129,38650,"Roissard, Grenoble, Isère, Auvergne-Rhône-Alpe...","(44.8949372121526, 5.628685069683022, 0.0)",NaN,Auvergne-Rhône-Alpes,France
1854,28729,"Cabanillas de la Sierra, Sierra Norte, Comunid...","(40.81087744705882, -3.6338493930013165, 0.0)",NaN,Comunidad de Madrid,España
7576,11649,"Maciachini M3, Piazzale Carlo Maciachini, Derg...","(45.4978436, 9.1847351, 0.0)",Milano,Lombardia,Italia
475,36370,"Vollore-Ville, Thiers, Puy-de-Dôme, Auvergne-R...","(45.794642536363625, 3.6289031620680094, 0.0)",NaN,Auvergne-Rhône-Alpes,France
8589,7350-258,"258, Ettaler Forst, Landkreis Garmisch-Partenk...","(47.522495, 10.9775009, 0.0)",Ettaler Forst,Bayern,Deutschland


In [131]:
df_cities.isna().sum()

zipcode        1
location       0
coords         0
city        5639
state       1244
country        5
dtype: int64

Hay algunas ciudades y comunidades que nos se han podido encontrar.

In [132]:
# La columna de location ya no es necesaria
df_cities.drop(columns=['location'],inplace=True)

In [133]:
df_items = df_items.merge(df_cities, on='zipcode', how='left')
df_items.sample(5,random_state=seed)

,num_order,item_id,created_at,product_id,qty_ordered,base_cost,price,discount_percent,customer_id,city_x,zipcode,coords,city_y,state,country
478335,07c4fc02f512b224b29df6351cda166a,78e531d943858f99c4c787bbf3278534,2018-04-19 12:19:58,2231,3,1.6225,3.27,5.0,7d4f924fc8c8b3d8db2735d93c7a7447,cádiz,11009,"(36.5297438, -6.2928976, 0.0)",Cadiz,Andalucía,España
701470,a63a34f61636547af0d5658f356372d2,b3337941c2fe2ba67215dfcf0c0dfdf0,2018-09-06 17:52:30,11068,3,12.7500,14.20,9.0,c4b29f82e8b02e4532aa30a054792e1f,palma de mallorca,07003,"(39.576480615517234, 2.649658752548201, 0.0)",Palma,Illes Balears,España
417946,565da574d7c9671b6b546fc40ca94d3c,5eb9196ce134003f17eb4b62ecb7d69a,2018-03-19 09:44:51,24172,1,9.2400,12.14,20.0,9da129a43d6efb66b5d32374b72ad2cc,getafe,28905,"(40.29327263947368, -3.7531575077857626, 0.0)",Getafe,Comunidad de Madrid,España
66625,7eebb4acfd3801b550f1fc5bfcc2c43c,6ff01f9a53af7ea093b931a8dafa2f99,2017-03-11 09:08:01,33995,1,8.6073,16.31,5.0,9bf88927e24ab526ed285da976feecf0,villacarrillo,23300,"(46.24211997210627, 1.4959725607293608, 0.0)",NaN,Nouvelle-Aquitaine,France
257014,a3e12c9601160b95f472385bbe2817cd,89489b0f4f734f9ca5afcd0de01be260,2017-10-18 13:38:58,3685,1,6.1866,8.66,10.0,44e504394d288b0c2617e70d8e9f58ab,madrid,28017,"(40.429118719300796, -3.6456800082218725, 0.0)",Madrid,Comunidad de Madrid,España


In [134]:
#Comprobamos que no se han añadidos filas con el merge
df_items.shape

(990868, 15)

In [135]:
df_items.isna().sum()

num_order                0
item_id                  0
created_at               0
product_id               0
qty_ordered              0
base_cost             2470
price                    0
discount_percent         0
customer_id              0
city_x                   0
zipcode                  0
coords                3257
city_y              454633
state                15071
country               3298
dtype: int64

Tenemos que manejar cuidadosamente los datos introducidos por las nuevas columnas.

In [136]:
# De las ciudades recuperadas las imputamos en la columna original
indexes = df_items[df_items['city_y'].notna()].index
cities = df_items[df_items['city_y'].notna()]['city_y'].to_list()
df_items.loc[indexes,'city_x'] = cities

In [137]:
# Las coordenadas que faltan las imputamos como vacío -> al no ser muchas no afectaran al mapa, ni a los filtros
indexes = df_items[df_items['coords'].isna()].index
df_items.loc[indexes, 'coords'] = df_items.loc[indexes, 'coords'].apply(lambda x: "('','','')")

In [138]:
df_items["coords"] = df_items["coords"].apply(lambda x: eval(x))

In [139]:
df_items['lat'], df_items['lon'], _ = zip(*df_items['coords'])

In [140]:
df_items['city_x'] = df_items['city_x'].str.capitalize()
df_items.rename(columns={'city_x': 'city'},inplace=True)
df_items.drop(columns=['city_y'],inplace=True)

In [141]:
df_items.isna().sum()

num_order               0
item_id                 0
created_at              0
product_id              0
qty_ordered             0
base_cost            2470
price                   0
discount_percent        0
customer_id             0
city                    0
zipcode                 0
coords                  0
state               15071
country              3298
lat                     0
lon                     0
dtype: int64

In [142]:
print(f"Recuento ciudad únicas: {df_items['city'].nunique()}")

Recuento ciudad únicas: 9227


Vemos como efectivamente, el número de ciudades ha disminuido tras normalizar el campo. Aun así esto no nos salva de las diferentes nomenclaturas que pueden existir. Por eso en este caso las coordenadas son más fiables.

## Duplicados

En un primer momento avistamos la presencia de registros duplicados, algo que en teoría no debería darse, ya que cada orden tiene un identificador y hora unívocos. 

In [143]:
df_items.duplicated(keep='first').sum()

48318

In [144]:
print(f'Filas totales: {df_items.shape[0]}')
df_items.drop_duplicates(inplace=True)
print(f'Filas totales tras eliminar duplicados: {df_items.shape[0]}')

Filas totales: 990868
Filas totales tras eliminar duplicados: 942550


## Fecha y Hora

Con el fin de facilitar posteriores consultas, hemos decidido separar los campos de fecha y hora

In [145]:
df_items['time'] = df_items['created_at'].dt.time
df_items['date'] = df_items['created_at'].dt.date
df_items.sample(5,random_state=seed)

,num_order,item_id,created_at,product_id,qty_ordered,base_cost,price,discount_percent,customer_id,city,zipcode,coords,state,country,lat,lon,time,date
402450,66891c4ecf782a359e0b83f50653dc21,1695334ff14b8f1e0314736e2c7e5400,2018-03-12 21:52:19,34781,3,2.8139,4.87,5.0,11584937864e247cfc02d45725d65e87,Madrid,28021,"(40.34637340456449, -3.706884919800276, 0.0)",Comunidad de Madrid,España,40.346373,-3.706885,21:52:19,2018-03-12
949376,567fcfa211a67abb95cc21cecc455f62,33c8c169131818246e4602e0da00d3af,2018-12-22 14:44:17,419,1,5.9400,8.02,5.0,5c63b1d6b6b94111a661b95e9cec0725,San andrés del rabanedo,24191,"(42.613980670588234, -5.60283824186383, 0.0)",Castilla y León,España,42.613981,-5.602838,14:44:17,2018-12-22
179958,cd4cfbab85cf72f68cba3fe6c93f78fb,7e660187bac7b6cec7806b9ab82eb986,2017-07-14 12:28:28,7093,1,17.1029,19.90,5.0,6235795718efd574d56decacfdd27044,Inca,07300,"(45.06072118207309, 4.803692606051856, 0.0)",Auvergne-Rhône-Alpes,France,45.060721,4.803693,12:28:28,2017-07-14
505351,4980183016d7618413249cdd9c0aa759,c15d771acf8e7159a6a2124ed876e266,2018-04-27 14:21:25,3982,1,14.5700,14.36,5.0,8c14679a75c45b11008e54f611706e95,Navarcles,08270,"(49.61887918586957, 4.487797519355239, 0.0)",Grand Est,France,49.618879,4.487798,14:21:25,2018-04-27
483160,2c8274477270bf61ff63b9695dc9fbd3,93e048c84faab9d0a483824d7b76615d,2018-04-20 23:47:04,61658,1,12.6738,15.38,5.0,7acb01b507502f0c4421c1fef52de49f,Llera,06227,"(, , )",NaN,NaN,,,23:47:04,2018-04-20


In [146]:
# Nos deshacemos de la columna original
df_items.drop(columns=['created_at'],inplace=True)

## Unificación de conjuntos

Con el objetivo de simplificar las consultas, se añadirán tres campos más a este conjunto de datos. A través del *product_id* se hará combinará las tablas de **items** con **products** trayendo los campos de *nombre de producto*, *marca* y *categoría*. Así se podrá ver si los pedidos están relacionados con alguna categoría, qué marcas se venden más, etc.

In [147]:
df_products.duplicated(subset=['product_id']).sum()

4646

De lo que nos percatamos es que los identificadores de productos no son únicos, sino que se asigna a diferentes productos. Esto dificulta el *merge*, ya que las claves están duplicada en una de las tablas. Para evitar este comportamiento hemos decidido quedarnos con los primeros id de productos que se encuentren.

In [148]:
df_products.drop_duplicates(subset=['product_id'],keep='first',inplace=True)

In [149]:
df_all = df_items.merge(df_products[['product_id','name','marca_value','analytic_category']],on='product_id',how='left')

In [150]:
# Comprobamos que se ha ejecutado correctamente
print(f'Tamaño tabla conjunta: {df_all.shape[0]}')
print(f'Tamaño tabla ordenes: {df_items.shape[0]}')

Tamaño tabla conjunta: 942550
Tamaño tabla ordenes: 942550


En las columnas nuevas, aparecen valores nulos. Esto se debe a que el producto no se ha encontrado, así que prescindiremos de dichas filas, ya que no tenemos modo alguno de recuperar dichos valores.

In [151]:
df_all.isna().sum()

num_order                0
item_id                  0
product_id               0
qty_ordered              0
base_cost             2390
price                    0
discount_percent         0
customer_id              0
city                     0
zipcode                  0
coords                   0
state                14570
country               3271
lat                      0
lon                      0
time                     0
date                     0
name                 46638
marca_value          46638
analytic_category    46638
dtype: int64

In [152]:
df_all.dropna(subset=['name','marca_value','analytic_category'],inplace=True)

## Precios

Lo primero de todo es que había bastantes valores nulos en la columna *base_cost*

In [153]:
df_all.isna().sum()

num_order                0
item_id                  0
product_id               0
qty_ordered              0
base_cost             1089
price                    0
discount_percent         0
customer_id              0
city                     0
zipcode                  0
coords                   0
state                13988
country               3082
lat                      0
lon                      0
time                     0
date                     0
name                     0
marca_value              0
analytic_category        0
dtype: int64

Veamos como recuperar alguno de los valores

In [154]:
# Obtenemos los id de productos sin precio base
df_pr = df_all[df_all['base_cost'].isna()]['product_id'].drop_duplicates()

In [155]:
# Buscamos aquellos product_id que si tengan base_cost para así recuperar sus valores
recoverable = df_all[(df_all['product_id'].isin(df_pr) ) & (df_all['base_cost'].notna())][['product_id','base_cost']]

# Imputaremos sus valores por la media
recoverable = recoverable.groupby('product_id').agg({'base_cost': np.mean}).reset_index()
recoverable

,product_id,base_cost
0,5363,13.696000
1,5367,6.416121
2,6070,0.000000
3,6152,11.910819
4,6179,3.945000
...,...,...
388,27216,2.639125
389,27237,5.154800
390,27516,1.672000
391,27666,16.406883


In [156]:
# Asignamos valores recuperados
for i,r in recoverable.iterrows():
    df_all.loc[df_all['product_id']==r['product_id'],'base_cost'] = r['base_cost']

Vemos que se reduce sustancialmente el número de nulos. Los registros restantes, al ser muy poco y no tener otra fuente de donde recuperar información, serán eliminadas

In [157]:
df_all.isna().sum()

num_order                0
item_id                  0
product_id               0
qty_ordered              0
base_cost               37
price                    0
discount_percent         0
customer_id              0
city                     0
zipcode                  0
coords                   0
state                13988
country               3082
lat                      0
lon                      0
time                     0
date                     0
name                     0
marca_value              0
analytic_category        0
dtype: int64

In [158]:
df_all.dropna(subset=['base_cost'],inplace=True)

Por otro lado, en esta misma columna hay valores negativos, lo cual es bastante extraño, ya que no se trata de una tabla donde su intención sea plasmar devoluciones. Lo que es más, si las consideramos como tales tampoco tendría sentido, ya que es el precio de coste, el cual no se debería ver afectado en dichas operaciones

In [159]:
# Obtenemos los id de productos con precios negativos
df_pr = df_all[df_all['base_cost'] < 0]['product_id'].drop_duplicates()
print(f'Producto con base_cost nulo: {df_pr.shape[0]}')

Producto con base_cost nulo: 12


In [160]:
# Buscamos en el resto de la tabla aquellos product_id que tengan base_cost positivo
recoverable = df_all[(df_all['product_id'].isin(df_pr)) & (df_all['base_cost'] > 0)][['product_id','base_cost']]

# Imputaremos sus valores por la media
recoverable = recoverable.groupby('product_id').agg({'base_cost': np.mean}).reset_index()
recoverable

,product_id,base_cost
0,3388,9.103685
1,6242,17.964623
2,7121,7.043431
3,7550,27.084500
4,8335,5.638369
5,11504,1.006107
6,39565,5.344010
7,42466,7.130660
8,51964,7.286638
9,61811,8.039493


In [161]:
# Asignamos valores recuperados
for i,r in recoverable.iterrows():
    df_all.loc[df_all['product_id']==r['product_id'],'base_cost'] = r['base_cost']

Por último, para evitar descuadres en los números se ha preferido redondear a dos decimales las columnas de precio.

In [162]:
df_all.loc[:,['base_cost','price']] = df_all[['base_cost','price']].round(decimals=2)

Con todas las operaciones ya hechas, lo único que queda es guardar el dataframe en formato *csv*

In [163]:
# Reordenamos las columnas según nuestro criterio
cols = ['num_order','item_id','customer_id','time','date','city','state','country','lat','lon','product_id','name','base_cost','price','discount_percent','qty_ordered','marca_value','analytic_category']
df_all = df_all[cols]

In [164]:
df_all.sample(5,random_state=seed)

,num_order,item_id,customer_id,time,date,city,state,country,lat,lon,product_id,name,base_cost,price,discount_percent,qty_ordered,marca_value,analytic_category
687904,25564d66c93732753f5751ea84b2d8a1,86236435c2b564babc323ddaeae26803,4590bb19321a6a701b93e1987c3cf540,21:29:58,2018-09-12,Lisboa,NaN,Portugal,38.744052,-9.151828,2607,xhekpon crema antiarrugas 40g,3.79,4.74,5.0,2,Xhekpon,Cosmética y Belleza
719183,eb0190a0fa2b0ae1126f1c38fb021089,c5145c65a458ce0fcc67cd761360caf1,ab1e132fade52231330f893f9d12ebf3,08:16:50,2018-10-01,Guilvinec,Bretagne,France,47.797603,-4.279368,11279,elifexir minucell emulsion tensora 200 ml,5.63,6.75,5.0,1,Elifexir,Cosmética y Belleza
658160,14b9cbab629647719d744731836da625,853f300e8bdca94b30a1920391819643,cc4c116b60527eff855a2ed9a8245060,13:55:09,2018-09-04,Manises,Comunitat Valenciana,España,39.493381,-0.468153,2452,bexident colutorio dientes sensibles 500ml,4.66,5.69,9.0,1,Bexident,Higiene
659752,7d761c3e10fe06c3b6b9ff11d446b203,abef041860f4bce3948d9177e150a96a,def133daf9dacfd81eaf7d0217d3829e,21:04:09,2018-09-04,Madrid,Piemonte,Italia,45.628506,8.536299,60968,juanola propolis regaliz 24 pastillas blandas,2.88,3.60,5.0,1,Juanola,Herbolario
790999,c289097d2ffd4f2cc9a5941d6d55f6f5,6abd538cc06f3172834bcf45fef9202e,a23d24fa680420edc0ce386d48dfaf2c,18:09:21,2018-11-05,San fernando,Occitanie,France,43.174835,3.003775,35205,saforelle gel intimo 500ml,6.90,9.31,8.0,1,Saforelle,Higiene


In [165]:
df_all.to_csv('./clean_items.csv',index=False)

## Conclusiones

De este cuaderno nos quedamos con las siguientes conclusiones:

- La importancia de normalizar los campos (ciudades y fechas)
- El campo *product_id* necesita ser complementado para evitar perder información
- Los precios perdidos de los productos se pueden recuperar facilmente